# Cofactor swap targets

The conversion of tryptamine to 4-hydroxy-tryptamine uses NADPH as a cofactor, and therefore a cofactor swap analysis was performed in order to determine whether increasing the pool of available NADPH by swapping the cofactor specificty of reactions in the model from NADPH to NADH could increase psilocybin production. This hypothesis is supported by research by King et al., which showed that swapping the cofactor specificity from NAD(H) to NADP(H) in oxidoreductase enzymes involved in central carbon metabolism can improve the theoretical yields of several amino acids including L-aspartate, L-lysine, and L-isoleucine as well as some non-native products in *S. cerevisiae* (King, Feist, 2014). CofactorSwapOptimization utilizes an algorithm that identifies which reactions in the model can undergo a swap in cofactor specificity in order to increase the availability of a cofactor that is rate-limiting for the production of a target metabolite, and increase its theoretical yield.

In [1]:
from cobra import Metabolite, Reaction
from cobra.io import read_sbml_model


In [2]:

model = read_sbml_model('/workspaces/group-assingment-team-10/Lipomyces_starkeyi.xml')

In [3]:
from collections.abc import Mapping

In [4]:


from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization, NADH_NADPH

ImportError: cannot import name 'find_gene_knockout_reactions' from 'cobra.manipulation.delete' (/workspaces/group-assingment-team-10/.conda/lib/python3.8/site-packages/cobra/manipulation/delete.py)

In [2]:
model.solver = "glpk" # what is this?

In [3]:
biomass = model.reactions.DRAIN_Biomass

In [4]:
biomass.lower_bound = 0.1 # OBS denne skal måske ikke være 0,1 tjek tutorial notebooken og forstå det


In [5]:
demand = model.reactions.DM_psi

The demand reaction for the "target" (psilocybin) is defined in "01-model.ipynb"

In [6]:
model.objective = demand

In [7]:
product_yield = product_yield(demand, model.reactions.BOUNDARY_GLCe)

In [8]:
CofactorSwapOptimization?

Init signature:
CofactorSwapOptimization(
    cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']),
    candidate_reactions=None,
    skip_reactions=None,
    *args,
    **kwargs,
)
Docstring:     
Optimize co-factor swapping

As suggested in [1]_, flux through a given reaction can sometimes be optimized by swapping complementary
co-factor. This class implements a search for reactions when swapped improve the given objective. Briefly,
the approach is to

- find reactions that have all the targeted co-factor pairs e.g. (nad_c -> nadp_c, nadh_c -> nadph_c)

- add reactions that have the co-factors swapped and then by a search algorithm switching one off in favor of the
  other

The implementation here differs from that in [1]_ in that we use a general purpose search algorithm rather than
formulating the search as a mixed integer linear programming problem.

References
----------
.. [1] King, Zachary A., and Adam M. Feist. "Optimizing Cofactor Specificity of Oxidoreductase Enzym

Our model has different syntax for the default cofactors, and therefore the parameter cofactor_id_swaps must be added to the CofactorSwapOptimization function input with the correct syntax. 

In [21]:
swap_opt = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True, cofactor_id_swaps=(['NADP', 'NADPH'], ['NAD', 'NADH']))

In [22]:
%%time
result = swap_opt.run(max_size=5)

Starting optimization at Tue, 01 Dec 2020 10:45:44


HBox()

Finished after 00:01:24
CPU times: user 2min 42s, sys: 85.9 ms, total: 2min 42s
Wall time: 2min 42s


In [23]:
result

,index,targets,fitness
0,0,"(r581,)",0.550228
1,1,"(r252,)",0.550228


In [24]:
model.reactions.r581

Reaction identifier,r581
Name,Enoyl-ACP reductase
Memory address,0x07f971801de10
Stoichiometry,C10DACP + NADPH --> C100ACP + NADP Dec-2-enoyl-[acyl-carrier protein] + NADPH --> Decanoyl-[acyl-carrier protein] + NADP+
GPR,1105220 or 1146027 or 1146734
Lower bound,0.0
Upper bound,1000.0


In [25]:
model.reactions.r252

Reaction identifier,r252
Name,NADPH oxidase
Memory address,0x07f9718283be0
Stoichiometry,NADPH + O2 --> H2O2 + NADP NADPH + Oxygen --> Hydrogen peroxide + NADP+
GPR,1155658
Lower bound,0.0
Upper bound,1000.0


The result of the cofactor swap optimizsation identified r581 and r252 as targets swapping cofactor specificity to improve theoretical yield of psilocybin. In reality, NADPH oxidase can only oxidize NADPH. Therefore r581 (Enoyl-ACP reductase) is the only meaningful result. 

## References

1) King, Z. and Feist, A., 2014. Optimal cofactor swapping can increase the theoretical yield for chemical production in Escherichia coli and Saccharomyces cerevisiae. Metabolic Engineering, 24, pp.117-128.